In [ ]:
!pip install torch
!pip install torch-geometric
!pip install torch_spline_conv
!pip install torch_scatter 
!pip install torch_cluster #slow
!pip install torch_sparse #slow
!pip install matplotlib
!pip install ogb
!pip install networkx

In [1]:
print(3)

3


In [2]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim
import random
from models import GCN

import torch_geometric

import torch_sparse

import matplotlib.pyplot as plt
from sklearn import metrics

from utils import get_plot


In [ ]:
from data_process import generate_data, load_data
from train_func import test, train, Lhop_Block_matrix_train

In [ ]:
def get_K_hop_neighbors(adj_matrix, index, K):
    adj_matrix = adj_matrix + torch.eye(adj_matrix.shape[0],adj_matrix.shape[1])  #make sure the diagonal part >= 1
    hop_neightbor_index=index
    for i in range(K):
        hop_neightbor_index=torch.unique(torch.nonzero(adj[hop_neightbor_index])[:,1])
    return hop_neightbor_index

In [ ]:
import scipy.sparse as sp
def normalize(mx):  #adj matrix
    
    mx = mx + torch.eye(mx.shape[0],mx.shape[1])
    
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return torch.tensor(mx)

# Model

In [ ]:
#define model

#for compare 2-10 layer performance in appendix
#iterations = 400
#Adam, lr = 0.01


def centralized_GCN(features, adj, labels, idx_train, idx_val, idx_test, num_layers):
    
        model = GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers)
        model.reset_parameters()
        if args_cuda:
                #from torch_geometric.nn import DataParallel
                #model = DataParallel(model)
                #model= torch.nn.DataParallel(model)
                model=model.cuda()
                
                #features= torch.nn.DataParallel(features)
                
                features = features.cuda()
                
                #edge_index= torch.nn.DataParallel(edge_index)
                
                adj = adj.cuda()
                labels = labels.cuda()
                idx_train = idx_train.cuda()
                idx_val = idx_val.cuda()
                idx_test = idx_test.cuda()
        #optimizer and train
        
        optimizer = optim.SGD(model.parameters(),
                              lr=args_lr, weight_decay=args_weight_decay)
        
        
        #optimizer = optim.Adam(model.parameters(),
        #                      lr=args_lr, weight_decay=args_weight_decay)
        # Train model
        best_val=0
        for t in range(args_iterations): #make to equivalent to federated
            loss_train, acc_train=train(t, model, optimizer, features, adj, labels, idx_train)
            # validation
            loss_train, acc_train= test(model, features, adj, labels, idx_train) #train after backward
            print(t,"train",loss_train,acc_train)
            loss_val, acc_val= test(model, features, adj, labels, idx_val) #validation
            print(t,"val",loss_val,acc_val)
            
            a = open(dataset_name+'_IID_'+'centralized_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations),'a+')
            a.write(str(t)+'\t'+"train"+'\t'+str(loss_train)+'\t'+str(acc_train)+'\n')
            a.write(str(t)+'\t'+"val"+'\t'+str(loss_val)+'\t'+str(acc_val)+'\n')
            a.close()
            
        #test  
        loss_test, acc_test= test(model, features, adj, labels, idx_test)
        print(t,'\t',"test",'\t',loss_test,'\t',acc_test)
        a = open(dataset_name+'_IID_'+'centralized_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations),'a+')
        a.write(str(t)+'\t'+"test"+'\t'+str(loss_test)+'\t'+str(acc_test)+'\n')
        a.close()
        
        print("save file as",dataset_name+'_IID_'+'centralized_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations))
        del model
        del features 
        del adj
        del labels
        del idx_train
        del idx_val
        del idx_test
        
        return loss_test, acc_test

In [ ]:
def setdiff1d(t1, t2):
    
    combined = torch.cat((t1, t2))
    uniques, counts = combined.unique(return_counts=True)
    difference = uniques[counts == 1]
    #intersection = uniques[counts > 1]
    return difference
    

In [ ]:
def intersect1d(t1, t2):
    
    combined = torch.cat((t1, t2))
    uniques, counts = combined.unique(return_counts=True)
    #difference = uniques[counts == 1]
    intersection = uniques[counts > 1]
    return intersection

In [ ]:
def BDS_federated_GCN(K, features, adj, labels, idx_train, idx_val, idx_test, iid_percent, sample_rate =0.5, L_hop=1, num_layers=2):
        # K: number of models
        #choose adj matrix
        #multilayer_GCN:n*n
        #define model
        global_model = GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers)
        global_model.reset_parameters()
        models=[]
        for i in range(K):
            models.append(GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers))
        if args_cuda:
                for i in range(K):
                    models[i]=models[i].cuda()
                global_model=global_model.cuda()
                features = features.cuda()
                adj = adj.cuda()
                labels = labels.cuda()
                idx_train = idx_train.cuda()
                idx_val = idx_val.cuda()
                idx_test = idx_test.cuda()
        #optimizer and train
        optimizers=[]
        for i in range(K):
            optimizers.append(optim.SGD(models[i].parameters(),
                              lr=args_lr, weight_decay=args_weight_decay))
        # Train model
        
        row, col, edge_attr = adj.t().coo()
        edge_index = torch.stack([row, col], dim=0)
        
        
        split_data_indexes=[]
        
        nclass=labels.max().item() + 1
        split_data_indexes = []
        non_iid_percent = 1 - float(iid_percent)
        iid_indexes = [] #random assign
        shuffle_labels = [] #make train data points split into different devices
        for i in range(K):
            current = torch.nonzero(labels == i).reshape(-1)
            current = current[np.random.permutation(len(current))] #shuffle
            shuffle_labels.append(current)
                
        average_device_of_class = K // nclass
        if K % nclass != 0: #for non-iid
            average_device_of_class += 1
        for i in range(K):  
            label_i= i // average_device_of_class    
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * non_iid_percent)
            split_data_indexes.append((labels_class[average_num * (i % average_device_of_class):average_num * (i % average_device_of_class + 1)]))
        
        if args_cuda:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))).cuda(), torch.cat(split_data_indexes))
        else:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))), torch.cat(split_data_indexes))
        
        iid_indexes = iid_indexes[np.random.permutation(len(iid_indexes))]
        
        for i in range(K):  #for iid
            label_i= i // average_device_of_class
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * (1 - non_iid_percent))
            split_data_indexes[i] = list(split_data_indexes[i]) + list(iid_indexes[:average_num])
                    
            iid_indexes = iid_indexes[average_num:]
            
        communicate_indexes = []
        in_com_train_data_indexes = []

        for i in range(K):
            if args_cuda:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i]).cuda()
            else:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i])
                
            split_data_indexes[i] = split_data_indexes[i].sort()[0]
            
            #communicate_index=get_K_hop_neighbors(adj, split_data_indexes[i], L_hop) #normalized adj
            
            communicate_index = torch_geometric.utils.k_hop_subgraph(split_data_indexes[i],L_hop,edge_index)[0]
            communicate_indexes.append(communicate_index)
            communicate_indexes[i] = communicate_indexes[i].sort()[0]
            
            inter = intersect1d(split_data_indexes[i], idx_train)  ###only count the train data of nodes in current server(not communicate nodes)   
            in_com_train_data_indexes.append(torch.searchsorted(communicate_indexes[i], inter).clone())   #local id in block matrix

            
            
        #assign global model weights to local models at initial step
        for i in range(K):
            models[i].load_state_dict(global_model.state_dict())
        
        for t in range(args_iterations):
            acc_trains=[]
            for i in range(K):
                for epoch in range(args_epochs):
                    diff = setdiff1d(split_data_indexes[i], communicate_indexes[i])
                    sample_index = torch.cat((split_data_indexes[i], diff[torch.randperm(len(diff))[:int(len(diff) * sample_rate)]])).clone()

                    sample_index = sample_index.sort()[0]
                    
                    inter = intersect1d(split_data_indexes[i], idx_train)  ###only count the train data of nodes in current server(not communicate nodes)
                    in_sample_train_data_index = torch.searchsorted(sample_index, inter).clone()   #local id in block matrix

                    if len(in_sample_train_data_index) == 0:
                        continue
                    try:
                        adj[sample_index][:,sample_index]
                    except: #adj is empty
                        continue
                    
                    acc_train = FedSage_train(epoch, models[i], optimizers[i], 
                                                        features, adj, labels, sample_index, in_sample_train_data_index)
                acc_trains.append(acc_train)
                
            states=[]
            gloabl_state=dict()
            for i in range(K):
                states.append(models[i].state_dict())
            # Average all parameters
            for key in global_model.state_dict():
                gloabl_state[key] = in_com_train_data_indexes[0].shape[0] * states[0][key]
                count_D=in_com_train_data_indexes[0].shape[0]
                for i in range(1,K):
                    gloabl_state[key] += in_com_train_data_indexes[i].shape[0] * states[i][key]
                    count_D += in_com_train_data_indexes[i].shape[0]
                gloabl_state[key] /= count_D

            global_model.load_state_dict(gloabl_state)
            
            # Testing
            
            loss_train, acc_train = test(global_model, features, adj, labels, idx_train)
            print(t,'\t',"train",'\t',loss_train,'\t',acc_train)
            
            loss_val, acc_val = test(global_model, features, adj, labels, idx_val) #validation
            print(t,'\t',"val",'\t',loss_val,'\t',acc_val)
            

            a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_BDS_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')

            
            a.write(str(t)+'\t'+"train"+'\t'+str(loss_train)+'\t'+str(acc_train)+'\n')
            a.write(str(t)+'\t'+"val"+'\t'+str(loss_val)+'\t'+str(acc_val)+'\n')
            a.close()
            for i in range(K):
                models[i].load_state_dict(gloabl_state)
        #test  
        loss_test, acc_test= test(global_model, features, adj, labels, idx_test)
        print(t,'\t',"test",'\t',loss_test,'\t',acc_test)
        a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_BDS_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')
        a.write(str(t)+'\t'+"test"+'\t'+str(loss_test)+'\t'+str(acc_test)+'\n')
        a.close()
        print("save file as",dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_BDS_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K))
        
        del global_model
        del features 
        del adj
        del labels
        del idx_train
        del idx_val
        del idx_test
        while(len(models)>=1):
            del models[0]
        
        return loss_test, acc_test



In [ ]:
def FedSage_plus(K, features, adj, labels, idx_train, idx_val, idx_test, iid_percent, L_hop = 1, num_layers=2):
        # K: number of models
        #choose adj matrix
        #multilayer_GCN:n*n
        #define model
        global_model = GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers)
        global_model.reset_parameters()
        models=[]
        for i in range(K):
            models.append(GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers))
        if args_cuda:
                for i in range(K):
                    models[i]=models[i].to(device)
                global_model=global_model.to(device)
                features = features.to(device)
                adj = adj.to(device)
                labels = labels.to(device)
                idx_train = idx_train.to(device)
                idx_val = idx_val.to(device)
                idx_test = idx_test.to(device)
        #optimizer and train
        optimizers=[]
        for i in range(K):
            optimizers.append(optim.SGD(models[i].parameters(),
                              lr=args_lr, weight_decay=args_weight_decay))
        # Train model
        
        row, col, edge_attr = adj.t().coo()
        edge_index = torch.stack([row, col], dim=0)
        
        
        split_data_indexes=[]
        
        nclass=labels.max().item() + 1
        split_data_indexes = []
        non_iid_percent = 1 - float(iid_percent)
        iid_indexes = [] #random assign
        shuffle_labels = [] #make train data points split into different devices
        for i in range(K):
            current = torch.nonzero(labels == i).reshape(-1)
            current = current[np.random.permutation(len(current))] #shuffle
            shuffle_labels.append(current)
                
        average_device_of_class = K // nclass
        if K % nclass != 0: #for non-iid
            average_device_of_class += 1
        for i in range(K):  
            label_i= i // average_device_of_class    
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * non_iid_percent)
            split_data_indexes.append((labels_class[average_num * (i % average_device_of_class):average_num * (i % average_device_of_class + 1)]))
        
        if args_cuda:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))).to(device), torch.cat(split_data_indexes))
        else:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))), torch.cat(split_data_indexes))
        
        iid_indexes = iid_indexes[np.random.permutation(len(iid_indexes))]
        
        for i in range(K):  #for iid
            label_i= i // average_device_of_class
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * (1 - non_iid_percent))
            split_data_indexes[i] = list(split_data_indexes[i]) + list(iid_indexes[:average_num])
                    
            iid_indexes = iid_indexes[average_num:]
            
        communicate_indexes = []
        in_com_train_data_indexes = []
        for i in range(K):
            if args_cuda:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i]).to(device)
            else:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i])
                
            split_data_indexes[i] = split_data_indexes[i].sort()[0]
            
            #communicate_index=get_K_hop_neighbors(adj, split_data_indexes[i], L_hop) #normalized adj
            
            communicate_index = torch_geometric.utils.k_hop_subgraph(split_data_indexes[i],L_hop,edge_index)[0]
                
            communicate_indexes.append(communicate_index)
            communicate_indexes[i] = communicate_indexes[i].sort()[0]
            
            inter = intersect1d(split_data_indexes[i], idx_train)  ###only count the train data of nodes in current server(not communicate nodes)

                
            in_com_train_data_indexes.append(torch.searchsorted(communicate_indexes[i], inter).clone())   #local id in block matrix
        
        features_in_clients = []
        #assume the linear generator learnt the optimal (the average of features of neighbor nodes)
        #gaussian noise
        for i in range(K):
            #orignial features of outside neighbors of nodes in client i
            original_feature_i = features[setdiff1d(split_data_indexes[i], communicate_indexes[i])].clone()
            
            gaussian_feature_i = original_feature_i + torch.normal(0, 0.1, original_feature_i.shape).to(device)
            
            copy_feature = features.clone()
            
            copy_feature[setdiff1d(split_data_indexes[i], communicate_indexes[i])] = gaussian_feature_i
            
            features_in_clients.append(copy_feature[communicate_indexes[i]])
            print(features_in_clients[i].shape, communicate_indexes[i].shape)

        #assign global model weights to local models at initial step
        for i in range(K):
            models[i].load_state_dict(global_model.state_dict())
        
        for t in range(args_iterations):
            acc_trains=[]
            for i in range(K):
                for epoch in range(args_epochs):
                    if len(in_com_train_data_indexes[i]) == 0:
                        continue
                    try:
                        adj[communicate_indexes[i]][:,communicate_indexes[i]]
                    except: #adj is empty
                        continue
                    acc_train = FedSage_train(epoch, models[i], optimizers[i], 
                                                        features_in_clients[i], adj, labels, communicate_indexes[i], in_com_train_data_indexes[i])
                    
                acc_trains.append(acc_train)
            states=[]
            gloabl_state=dict()
            for i in range(K):
                states.append(models[i].state_dict())
            # Average all parameters
            for key in global_model.state_dict():
                gloabl_state[key] = in_com_train_data_indexes[0].shape[0] * states[0][key]
                count_D=in_com_train_data_indexes[0].shape[0]
                for i in range(1,K):
                    gloabl_state[key] += in_com_train_data_indexes[i].shape[0] * states[i][key]
                    count_D += in_com_train_data_indexes[i].shape[0]
                gloabl_state[key] /= count_D

            global_model.load_state_dict(gloabl_state)
            
            # Testing
            
            loss_train, acc_train = test(global_model, features, adj, labels, idx_train)
            print(t,'\t',"train",'\t',loss_train,'\t',acc_train)
            
            loss_val, acc_val = test(global_model, features, adj, labels, idx_val) #validation
            print(t,'\t',"val",'\t',loss_val,'\t',acc_val)
            

            a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_FedSage_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')

            
            a.write(str(t)+'\t'+"train"+'\t'+str(loss_train)+'\t'+str(acc_train)+'\n')
            a.write(str(t)+'\t'+"val"+'\t'+str(loss_val)+'\t'+str(acc_val)+'\n')
            a.close()
            for i in range(K):
                models[i].load_state_dict(gloabl_state)
        #test  
        loss_test, acc_test= test(global_model, features, adj, labels, idx_test)
        print(t,'\t',"test",'\t',loss_test,'\t',acc_test)
        a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_FedSage_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')
        a.write(str(t)+'\t'+"test"+'\t'+str(loss_test)+'\t'+str(acc_test)+'\n')
        a.close()
        print("save file as",dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_FedSage_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K))
        
        del global_model
        del features 
        del adj
        del labels
        del idx_train
        del idx_val
        del idx_test
        while(len(models)>=1):
            del models[0]
        
        return loss_test, acc_test



In [ ]:
def Lhop_Block_federated_GCN(K, features, adj, labels, idx_train, idx_val, idx_test, iid_percent, L_hop, num_layers):
        # K: number of models
        #choose adj matrix
        #multilayer_GCN:n*n
        #define model
        global_model = GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers)
        global_model.reset_parameters()
        models=[]
        for i in range(K):
            models.append(GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers))
        if args_cuda:
                for i in range(K):
                    models[i]=models[i].cuda()
                global_model=global_model.cuda()
                features = features.cuda()
                adj = adj.cuda()
                labels = labels.cuda()
                idx_train = idx_train.cuda()
                idx_val = idx_val.cuda()
                idx_test = idx_test.cuda()
        #optimizer and train
        optimizers=[]
        for i in range(K):
            optimizers.append(optim.SGD(models[i].parameters(),
                              lr=args_lr, weight_decay=args_weight_decay))
        # Train model
        
        row, col, edge_attr = adj.t().coo()
        edge_index = torch.stack([row, col], dim=0)
        
        
        split_data_indexes=[]
        
        nclass=labels.max().item() + 1
        split_data_indexes = []
        non_iid_percent = 1 - float(iid_percent)
        iid_indexes = [] #random assign
        shuffle_labels = [] #make train data points split into different devices
        for i in range(K):
            current = torch.nonzero(labels == i).reshape(-1)
            current = current[np.random.permutation(len(current))] #shuffle
            shuffle_labels.append(current)
                
        average_device_of_class = K // nclass
        if K % nclass != 0: #for non-iid
            average_device_of_class += 1
        for i in range(K):  
            label_i= i // average_device_of_class    
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * non_iid_percent)
            split_data_indexes.append((labels_class[average_num * (i % average_device_of_class):average_num * (i % average_device_of_class + 1)]))
        
        if args_cuda:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))).cuda(), torch.cat(split_data_indexes))
        else:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))), torch.cat(split_data_indexes))
        iid_indexes = iid_indexes[np.random.permutation(len(iid_indexes))]
        
        for i in range(K):  #for iid
            label_i= i // average_device_of_class
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * (1 - non_iid_percent))
            split_data_indexes[i] = list(split_data_indexes[i]) + list(iid_indexes[:average_num])
                    
            iid_indexes = iid_indexes[average_num:]
            
        communicate_indexes = []
        in_com_train_data_indexes = []
        for i in range(K):
            if args_cuda:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i]).cuda()
            else:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i])
                
            split_data_indexes[i] = split_data_indexes[i].sort()[0]
            
            #communicate_index=get_K_hop_neighbors(adj, split_data_indexes[i], L_hop) #normalized adj
            
            communicate_index = torch_geometric.utils.k_hop_subgraph(split_data_indexes[i],L_hop,edge_index)[0]
                
            communicate_indexes.append(communicate_index)
            communicate_indexes[i] = communicate_indexes[i].sort()[0]
            
            inter = intersect1d(split_data_indexes[i], idx_train)  ###only count the train data of nodes in current server(not communicate nodes)

                
            in_com_train_data_indexes.append(torch.searchsorted(communicate_indexes[i], inter).clone())   #local id in block matrix

        #assign global model weights to local models at initial step
        for i in range(K):
            models[i].load_state_dict(global_model.state_dict())
        
        for t in range(args_iterations):
            acc_trains=[]
            for i in range(K):
                for epoch in range(args_epochs):
                    if len(in_com_train_data_indexes[i]) == 0:
                        continue
                    
                    try:
                        adj[communicate_indexes[i]][:,communicate_indexes[i]]
                    except: #adj is empty
                        continue
                    acc_train = Lhop_Block_matrix_train(epoch, models[i], optimizers[i], 
                                                        features, adj, labels, communicate_indexes[i], in_com_train_data_indexes[i])
                    
                acc_trains.append(acc_train)
            states=[]
            gloabl_state=dict()
            for i in range(K):
                states.append(models[i].state_dict())
            # Average all parameters
            for key in global_model.state_dict():
                gloabl_state[key] = in_com_train_data_indexes[0].shape[0] * states[0][key]
                count_D=in_com_train_data_indexes[0].shape[0]
                for i in range(1,K):
                    gloabl_state[key] += in_com_train_data_indexes[i].shape[0] * states[i][key]
                    count_D += in_com_train_data_indexes[i].shape[0]
                gloabl_state[key] /= count_D

            global_model.load_state_dict(gloabl_state)
            
            # Testing
            
            loss_train, acc_train = test(global_model, features, adj, labels, idx_train)
            print(t,'\t',"train",'\t',loss_train,'\t',acc_train)
            
            loss_val, acc_val = test(global_model, features, adj, labels, idx_val) #validation
            print(t,'\t',"val",'\t',loss_val,'\t',acc_val)
            

            a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_Block_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')

            
            a.write(str(t)+'\t'+"train"+'\t'+str(loss_train)+'\t'+str(acc_train)+'\n')
            a.write(str(t)+'\t'+"val"+'\t'+str(loss_val)+'\t'+str(acc_val)+'\n')
            a.close()
            for i in range(K):
                models[i].load_state_dict(gloabl_state)
        #test  
        loss_test, acc_test= test(global_model, features, adj, labels, idx_test)
        print(t,'\t',"test",'\t',loss_test,'\t',acc_test)
        a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_Block_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')
        a.write(str(t)+'\t'+"test"+'\t'+str(loss_test)+'\t'+str(acc_test)+'\n')
        a.close()
        print("save file as",dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_Block_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K))
        
        del global_model
        del features 
        del adj
        del labels
        del idx_train
        del idx_val
        del idx_test
        while(len(models)>=1):
            del models[0]
        
        return loss_test, acc_test



In [19]:
import torch
import random
import sys
import pickle as pkl
import numpy as np
import scipy.sparse as sp
#from scipy.sparse.linalg.eigen.arpack import eigsh
import networkx as nx
import torch_geometric
import torch_sparse

def generate_data(number_of_nodes, class_num, link_inclass_prob, link_outclass_prob): # 模拟用的
    
    
    adj=torch.zeros(number_of_nodes,number_of_nodes) #n*n adj matrix

    labels=torch.randint(0,class_num,(number_of_nodes,)) #assign random label with equal probability
    labels=labels.to(dtype=torch.long)
    #label_node, speed up the generation of edges
    label_node_dict=dict()

    for j in range(class_num):
            label_node_dict[j]=[]

    for i in range(len(labels)):
        label_node_dict[int(labels[i])]+=[int(i)]
    # label_id -> [client_id ... ....]

    #generate graph
    for node_id in range(number_of_nodes):
                j=labels[node_id]
                for l in label_node_dict:
                    if l==j: # 属于一个class
                        for z in label_node_dict[l]:  #z>node_id,  symmetrix matrix, no repeat
                            if z>node_id and random.random()<link_inclass_prob:
                                adj[node_id,z]= 1
                                adj[z,node_id]= 1
                    else:    # 属于不同class
                        for z in label_node_dict[l]:
                            if z>node_id and random.random()<link_outclass_prob:
                                adj[node_id,z]= 1
                                adj[z,node_id]= 1
                              
    adj=torch_geometric.utils.dense_to_sparse(torch.tensor(adj))[0]

    #generate feature use eye matrix
    features=torch.eye(number_of_nodes,number_of_nodes)
    
    
    

    #seprate train,val,test
    idx_train = torch.LongTensor(range(number_of_nodes//5))
    idx_val = torch.LongTensor(range(number_of_nodes//5, number_of_nodes//2))
    idx_test = torch.LongTensor(range(number_of_nodes//2, number_of_nodes))



    return features.float(), adj, labels, idx_train, idx_val, idx_test
    


def parse_index_file(filename):
    """Parse index file."""
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index
  
    

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def load_data(dataset_str):
    """
    Loads input data from gcn/data directory

    ind.dataset_str.x => the feature vectors of the training instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.tx => the feature vectors of the test instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.allx => the feature vectors of both labeled and unlabeled training instances
        (a superset of ind.dataset_str.x) as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.y => the one-hot labels of the labeled training instances as numpy.ndarray object;
    ind.dataset_str.ty => the one-hot labels of the test instances as numpy.ndarray object;
    ind.dataset_str.ally => the labels for instances in ind.dataset_str.allx as numpy.ndarray object;
    ind.dataset_str.graph => a dict in the format {index: [index_of_neighbor_nodes]} as collections.defaultdict
        object;
    ind.dataset_str.test.index => the indices of test instances in graph, for the inductive setting as list object.

    All objects above must be saved using python pickle module.

    :param dataset_str: Dataset name
    :return: All data input files loaded (as well the training/test data).
    """
    if dataset_str in ['cora', 'citeseer', 'pubmed']:
        names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
        objects = []
        for i in range(len(names)):
            with open("data/ind.{}.{}".format(dataset_str, names[i]), 'rb') as f:
                if sys.version_info > (3, 0):
                    objects.append(pkl.load(f, encoding='latin1'))
                else:
                    objects.append(pkl.load(f))

        x, y, tx, ty, allx, ally, graph = tuple(objects)
        print(type(x))
        print(x.shape)
        test_idx_reorder = parse_index_file("data/ind.{}.test.index".format(dataset_str))
        # print(f"test_idx_reorder:{test_idx_reorder}") # 应该那些需要预测的文章id
        test_idx_range = np.sort(test_idx_reorder)
        print(f"test_idx_range:{test_idx_range}") # [1708，2707]，预测1000篇文章 
        
        if dataset_str == 'citeseer':
            # Fix citeseer dataset (there are some isolated nodes in the graph)
            # Find isolated nodes, add them as zero-vecs into the right position
            test_idx_range_full = range(min(test_idx_reorder), max(test_idx_reorder)+1)
            tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
            tx_extended[test_idx_range-min(test_idx_range), :] = tx
            tx = tx_extended
            ty_extended = np.zeros((len(test_idx_range_full), y.shape[1]))
            ty_extended[test_idx_range-min(test_idx_range), :] = ty
            ty = ty_extended

        features = sp.vstack((allx, tx)).tolil() # 按行堆叠，其他维度需要相同
        features[test_idx_reorder, :] = features[test_idx_range, :] # 
        adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))

        labels = np.vstack((ally, ty)) # y
        labels[test_idx_reorder, :] = labels[test_idx_range, :]

        number_of_nodes=adj.shape[0]


        idx_test = test_idx_range.tolist()
        idx_train = range(len(y))
        idx_val = range(len(y), len(y)+500)

        idx_train = torch.LongTensor(idx_train)
        idx_val = torch.LongTensor(idx_val)
        idx_test = torch.LongTensor(idx_test)

        #features = normalize(features) #cannot converge if use SGD, why??????????
        #adj = normalize(adj)    # no normalize adj here, normalize it in the training process


        features=torch.tensor(features.toarray()).float()
        adj = torch.tensor(adj.toarray())
        adj = torch_sparse.tensor.SparseTensor.from_edge_index(torch_geometric.utils.dense_to_sparse(adj)[0])
        #edge_index=torch_geometric.utils.dense_to_sparse(torch.tensor(adj.toarray()))[0]
        labels=torch.tensor(labels)
        labels=torch.argmax(labels,dim=1)
    elif dataset_str in ['ogbn-arxiv', 'ogbn-products', 'ogbn-mag', 'ogbn-papers100M']: #'ogbn-mag' is heteregeneous
        #from ogb.nodeproppred import NodePropPredDataset
        from ogb.nodeproppred import PygNodePropPredDataset

        # Download and process data at './dataset/.'

        #dataset = NodePropPredDataset(name = dataset_str, root = 'dataset/')
        dataset = PygNodePropPredDataset(name='ogbn-arxiv',
                                     transform=torch_geometric.transforms.ToSparseTensor())

        split_idx = dataset.get_idx_split()
        idx_train, idx_val, idx_test = split_idx["train"], split_idx["valid"], split_idx["test"]
        idx_train = torch.LongTensor(idx_train)
        idx_val = torch.LongTensor(idx_val)
        idx_test = torch.LongTensor(idx_test)
        data = dataset[0]
        
        features = data.x #torch.tensor(graph[0]['node_feat'])
        labels = data.y.reshape(-1) #torch.tensor(graph[1].reshape(-1))
        adj = data.adj_t.to_symmetric()
        #edge_index = torch.tensor(graph[0]['edge_index'])
        #adj = torch_geometric.utils.to_dense_adj(torch.tensor(graph[0]['edge_index']))[0]

    return features.float(), adj, labels, idx_train, idx_val, idx_test

np.random.seed(42)
torch.manual_seed(42)
#'cora', 'citeseer', 'pubmed' #simulate #other dataset twitter, 
dataset_name="cora"#'ogbn-arxiv'

if dataset_name == 'simulate':
    number_of_nodes=200
    class_num=3
    link_inclass_prob=10/number_of_nodes  #when calculation , remove the link in itself
    #EGCN good when network is dense 20/number_of_nodes  #fails when network is sparse. 20/number_of_nodes/5

    link_outclass_prob=link_inclass_prob/20


    features, adj, labels, idx_train, idx_val, idx_test = generate_data(number_of_nodes,  class_num, link_inclass_prob, link_outclass_prob)               
else:
    features, adj, labels, idx_train, idx_val, idx_test = load_data(dataset_name)
    class_num = labels.max().item() + 1

# print(type(features))
# print(features.shape)
indices = torch.nonzero(features)
row = indices[:,0]
col = indices[:,1]

# print(f"row:{row}")
# print(f"col:{col}")
# print(features) # 每篇文章出现了哪些单词





/tmp/ipykernel_1823307/1302843733.py:107: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  objects.append(pkl.load(f, encoding='latin1'))


<class 'scipy.sparse._csr.csr_matrix'>
(140, 1433)
test_idx_range:[1708 1709 1710 1711 1712 1713 1714 1715 1716 1717 1718 1719 1720 1721
 1722 1723 1724 1725 1726 1727 1728 1729 1730 1731 1732 1733 1734 1735
 1736 1737 1738 1739 1740 1741 1742 1743 1744 1745 1746 1747 1748 1749
 1750 1751 1752 1753 1754 1755 1756 1757 1758 1759 1760 1761 1762 1763
 1764 1765 1766 1767 1768 1769 1770 1771 1772 1773 1774 1775 1776 1777
 1778 1779 1780 1781 1782 1783 1784 1785 1786 1787 1788 1789 1790 1791
 1792 1793 1794 1795 1796 1797 1798 1799 1800 1801 1802 1803 1804 1805
 1806 1807 1808 1809 1810 1811 1812 1813 1814 1815 1816 1817 1818 1819
 1820 1821 1822 1823 1824 1825 1826 1827 1828 1829 1830 1831 1832 1833
 1834 1835 1836 1837 1838 1839 1840 1841 1842 1843 1844 1845 1846 1847
 1848 1849 1850 1851 1852 1853 1854 1855 1856 1857 1858 1859 1860 1861
 1862 1863 1864 1865 1866 1867 1868 1869 1870 1871 1872 1873 1874 1875
 1876 1877 1878 1879 1880 1881 1882 1883 1884 1885 1886 1887 1888 1889
 1890 1891 

In [ ]:
if dataset_name in ['simulate', 'cora', 'citeseer', 'pubmed']:
    args_hidden = 16
else:
    args_hidden = 256

args_dropout = 0.5
args_lr = 1.0
args_weight_decay = 5e-4     #L2 penalty
args_epochs = 3
args_no_cuda = False
args_cuda = not args_no_cuda and torch.cuda.is_available()

args_device_num = class_num #split data into args_device_num parts
args_iterations = 500

In [ ]:
#for testing
centralized_GCN(features, adj, labels, idx_train, idx_val, idx_test, num_layers = 2)

In [ ]:

for i in range(10):
    centralized_GCN(features, adj, labels, idx_train, idx_val, idx_test, num_layers = 2)
    
for args_epochs in [3]:
    for args_random_assign in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        for i in range(10):
            Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 0, num_layers = 2)
            BDS_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign)
            Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 1, num_layers = 2)
            Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 2, num_layers = 2)

In [ ]:
for num_l in range(1, 11):
    for i in range(10):#10 times
        centralized_GCN(features, adj, labels, idx_train, idx_val, idx_test, num_layers = num_l)

In [ ]:
for args_epochs in [3]:
    for args_random_assign in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        for num_l in range(2, 11):
            for i in range(10):
                Block_federated_multilayer_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, num_layers = num_l)
                Lhop_Block_federated_multilayer_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 1, num_layers = num_l)
                Lhop_Block_federated_multilayer_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 2, num_layers = num_l)